# 1) Installation and Setup

In [1]:
import tensorflow as tf
print(tf.__version__)

2.4.1


In [2]:
import string  # for punctuation
import requests # to get the file in colab

# 2) Data Preprocessing

## Importing the dataset

In [3]:
# read the text directly from internet
response = requests.get('http://ocw.mit.edu/ans7870/6/6.006/s08/lecturenotes/files/t8.shakespeare.txt')

In [4]:
 response.text

Output hidden; open in https://colab.research.google.com to view.

In [5]:
data = response.text.split('\n')
# list of strings

In [6]:
data[0] # first line

'This is the 100th Etext file presented by Project Gutenberg, and'

In [7]:
data[253]

'  From fairest creatures we desire increase,'

In [8]:
# The real work of shakespare begins
data = data[253:]

In [9]:
# First line of shakespeare creation
data[0]

'  From fairest creatures we desire increase,'

In [10]:
len(data)
# 124k lines in data

124204

In [11]:
# join all the data to become a text
data = " ".join(data)

In [12]:
data # data without \n and previous comments

Output hidden; open in https://colab.research.google.com to view.

## Remove punctuations

In [13]:
# Function to clean the data
def clean_text(doc):
  tokens = doc.split()
  table = str.maketrans('','',string.punctuation)  # remove the punctuations
  tokens = [(w.translate(table)) for w in tokens] # list without punctuations
  tokens = [word for word in tokens if word.isalpha()] # remove non alphanumeric special characters
  tokens = [word.lower() for word in tokens]   # lowercase
  return tokens

In [14]:
tokens = clean_text(data)

In [15]:
print(tokens[:50])

['from', 'fairest', 'creatures', 'we', 'desire', 'increase', 'that', 'thereby', 'beautys', 'rose', 'might', 'never', 'die', 'but', 'as', 'the', 'riper', 'should', 'by', 'time', 'decease', 'his', 'tender', 'heir', 'might', 'bear', 'his', 'memory', 'but', 'thou', 'contracted', 'to', 'thine', 'own', 'bright', 'eyes', 'feedst', 'thy', 'lights', 'flame', 'with', 'selfsubstantial', 'fuel', 'making', 'a', 'famine', 'where', 'abundance', 'lies', 'thy']


In [16]:
len(tokens)

898199

In [17]:
# unique words
len(set(tokens))

27956

## Create data sequence

In [18]:
# cretae data seqience to train the model
# use particular set of words to predict the next word
# use 50 words to predict the next one  

In [19]:
length = 50+1 # 50 is input and +1 is output
lines = []

for i in range(length, len(tokens)):  # from 51 to len(tokens):
  seq = tokens[i-length: i]  # 0 to 51 the first sequence
  line = ' '.join(seq)  # create a line
  lines.append(line)  # 1 line = 51 words
  if i > 200000:   # take the firsts 200K words to train. Reducing time
    break

In [20]:
# total number of sentences
print(len(lines))

199951


In [21]:
lines[0] # first  51 words

'from fairest creatures we desire increase that thereby beautys rose might never die but as the riper should by time decease his tender heir might bear his memory but thou contracted to thine own bright eyes feedst thy lights flame with selfsubstantial fuel making a famine where abundance lies thy self'

In [22]:
tokens[0], tokens[50] # first and last word from lines[0]

('from', 'self')

In [23]:
lines[1]

'fairest creatures we desire increase that thereby beautys rose might never die but as the riper should by time decease his tender heir might bear his memory but thou contracted to thine own bright eyes feedst thy lights flame with selfsubstantial fuel making a famine where abundance lies thy self thy'

In [24]:
tokens[1], tokens[51] # first and last word from lines[1]

('fairest', 'thy')

## Tokenization

In [25]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [26]:
# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
# the sequence of words will be embedded as an integer (word embedding)
# convert text data into numerical values, using word embedding
sequences = tokenizer.texts_to_sequences(lines) # list of int values

In [27]:
# convert array to numpy array
sequences = np.array(sequences)

In [28]:
sequences[0]

array([   47,  1408,  1264,    37,   451,  1406,     9,  2766,  1158,
        1213,   171,   132,   269,    20,    24,     1,  4782,    87,
          30,    98,  4781,    18,   715,  1263,   171,   211,    18,
         829,    20,    27,  3807,     4,   214,   121,  1212,   153,
       13004,    31,  2765,  1847,    16, 13003, 13002,   754,     7,
        3806,    99,  2430,   466,    31,   307])

## Create x and y

In [29]:
# sequence is a two dimentional array
# first 50 columns is x
# the last one(51) is y (the value that we want)
x, y = sequences[:, :-1], sequences[:,-1]

In [30]:
x[0] # first line vector
# each number is a assinged in a unique word

array([   47,  1408,  1264,    37,   451,  1406,     9,  2766,  1158,
        1213,   171,   132,   269,    20,    24,     1,  4782,    87,
          30,    98,  4781,    18,   715,  1263,   171,   211,    18,
         829,    20,    27,  3807,     4,   214,   121,  1212,   153,
       13004,    31,  2765,  1847,    16, 13003, 13002,   754,     7,
        3806,    99,  2430,   466,    31])

In [31]:
x[0].shape

(50,)

In [32]:
y[0]

307

In [33]:
# index of each word
tokenizer.word_index

{'the': 1,
 'and': 2,
 'i': 3,
 'to': 4,
 'of': 5,
 'you': 6,
 'a': 7,
 'my': 8,
 'that': 9,
 'in': 10,
 'is': 11,
 'not': 12,
 'it': 13,
 'for': 14,
 'me': 15,
 'with': 16,
 'he': 17,
 'his': 18,
 'your': 19,
 'but': 20,
 'be': 21,
 'this': 22,
 'have': 23,
 'as': 24,
 'him': 25,
 'so': 26,
 'thou': 27,
 'will': 28,
 'what': 29,
 'by': 30,
 'thy': 31,
 'no': 32,
 'are': 33,
 'all': 34,
 'her': 35,
 'do': 36,
 'we': 37,
 'if': 38,
 'our': 39,
 'or': 40,
 'shall': 41,
 'thee': 42,
 'which': 43,
 'on': 44,
 'lord': 45,
 'o': 46,
 'from': 47,
 'good': 48,
 'more': 49,
 'sir': 50,
 'was': 51,
 'they': 52,
 'well': 53,
 'at': 54,
 'would': 55,
 'when': 56,
 'now': 57,
 'come': 58,
 'love': 59,
 'th': 60,
 'than': 61,
 'am': 62,
 'then': 63,
 'she': 64,
 'their': 65,
 'them': 66,
 'how': 67,
 'enter': 68,
 'let': 69,
 'did': 70,
 'ill': 71,
 'hath': 72,
 'one': 73,
 'us': 74,
 'know': 75,
 'first': 76,
 'make': 77,
 'had': 78,
 'like': 79,
 'here': 80,
 'upon': 81,
 'there': 82,
 'man': 83,


In [34]:
# length of total vocabulary
len(tokenizer.word_index)

13008

In [35]:
vocab_size = len(tokenizer.word_index) + 1  # +1 because starts in 0

In [36]:
vocab_size

13009

In [37]:
# in original text, total number of unique words are
len(set(tokens))
# after data cleaning it is reduce almost 13009

27956

In [38]:
y = to_categorical(y, num_classes=vocab_size)

In [39]:
# sequence lenght that we have
x.shape[1]

50

In [40]:
seq_length = x.shape[1]

In [41]:
seq_length

50

# 3) Building the LSTM Model

In [42]:
model = Sequential()

# First embedding layer
model.add(Embedding(input_dim=vocab_size, output_dim=50, input_length=seq_length))

# First LSTM layer
model.add(LSTM(units=100, return_sequences=True))

# Second LSTM layer
model.add(LSTM(units=100))

# Dense layer
model.add(Dense(units=100, activation='relu'))

# Final layer
model.add(Dense(units=vocab_size, activation='softmax'))

In [43]:
# summary of model
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 50)            650450    
_________________________________________________________________
lstm (LSTM)                  (None, 50, 100)           60400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 100)               10100     
_________________________________________________________________
dense_1 (Dense)              (None, 13009)             1313909   
Total params: 2,115,259
Trainable params: 2,115,259
Non-trainable params: 0
_________________________________________________________________


In [44]:
# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [45]:
# Train the model
model.fit(x,y, batch_size=256, epochs=100)

Epoch 1/100
782/782 [==============================] - 332s 420ms/step - loss: 7.1971 - accuracy: 0.0285
Epoch 2/100
782/782 [==============================] - 331s 424ms/step - loss: 6.5413 - accuracy: 0.0424
Epoch 3/100
782/782 [==============================] - 332s 424ms/step - loss: 6.3428 - accuracy: 0.0560
Epoch 4/100
782/782 [==============================] - 332s 424ms/step - loss: 6.1635 - accuracy: 0.0677
Epoch 5/100
782/782 [==============================] - 332s 425ms/step - loss: 6.0344 - accuracy: 0.0756
Epoch 6/100
782/782 [==============================] - 330s 422ms/step - loss: 5.9095 - accuracy: 0.0854
Epoch 7/100
782/782 [==============================] - 330s 422ms/step - loss: 5.8093 - accuracy: 0.0940
Epoch 8/100
782/782 [==============================] - 330s 422ms/step - loss: 5.7137 - accuracy: 0.1001
Epoch 9/100
782/782 [==============================] - 331s 423ms/step - loss: 5.6888 - accuracy: 0.1013
Epoch 10/100
782/782 [==============================] -

In [46]:
lines[12343]
# put this line as seed text

'home of love if i have ranged like him that travels i return again just to the time not with the time exchanged so that my self bring water for my stain never believe though in my nature reigned all frailties that besiege all kinds of blood that it could so'

In [47]:
seed_text = lines[12343]

ML model will predict the word on the basis of what it learned


In [50]:
# define a function

def generate_text_seq(model, tokenizer, text_seq_length, seed_text, n_words):
  text = []

  for _ in range(n_words):
    encoded = tokenizer.texts_to_sequences([seed_text]) [0] # 0 is for zeroth dimention of array
    encoded = pad_sequences([encoded], maxlen=text_seq_length, truncating='pre')

    y_predict = model.predict_classes(encoded)

    predicted_word = ''
    for word, index in tokenizer.word_index.items():
      if index == y_predict:
        predicted_word = word
        break
    seed_text = seed_text + ' ' + predicted_word
    text.append(predicted_word)
  return ' '.join(text)

In [51]:
generate_text_seq(model, tokenizer, seq_length, seed_text, 10)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


'preposterously be stained to leave to me and i will'